# Using Vertex AI to train an image classification model

<div class="alert alert-block alert-info">
Run the <a href="xxx"><code>00_pcam_setup.ipynb notebook</code></a> first, before running this one.  You'll need the settings info from that notebook.
</div>

## Introduction

This notebook shows some examples of how to use [Vertex AI](https://cloud.google.com/vertex-ai/docs) for training a machine learning model. 

Notebook `02_1_vertex_ai_pcam` showed how to define and submit a **model training job**; then how to upload and deploy that model for serving; and then how to send prediction requests to the deployed model's *Endpoint*. It also showed how to create and use a Managed Tensorboard instance during training, and how to log information about the training run to the Vertex Experiments API.

This notebook shows how to set up a [**hyperparameter tuning**](https://en.wikipedia.org/wiki/Hyperparameter_optimization) job using that same model; and how to set up and run a **distributed multi-node training** job.

Then, a following set of notebooks show how to use Vertex Pipelines to define an ML workflow for data preprocessing, training, model evaluation, and deployment.

The example code is here: https://github.com/verily-src/terra-solutions-ml.

### Estimated cost of running this notebook

The dataset used for the examples in this notebook is fairly large, as is the base model architecture, and training using the notebook's default configurations will take a number of hours. Time estimates are added in each section.

The model training works best with GPU(s)— it runs fine using only CPUs, but training will take an even longer time. For this example, the notebook itself doesn't need GPUs; instead they'll be used by Vertex AI.

The HP tuning example should cost ~37USD in Vertex AI charges to run (billed to your ['native' GCP project](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra)), and the distributed training example < 7.5USD in Vertex AI charges, not including the cost of the notebook instance.

### Running on a [Terra](http://app.terra.bio) notebook

This example requires that TensorFlow >= 2.6 be installed, and does not require GPUs; instead the example uses GPUs on Vertex AI Training.
You can use the default GATK image.

You will need to use a ['native' GCP project](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra) to connect to the Vertex AI services.  The `00_pcam_setup.ipynb` notebook, which should be run before this one, will walk you through that setup.

<div class="alert alert-block alert-info">
If you like, you can shut down the notebook instance/Cloud Environment while the training job runs— monitoring its progress in the Cloud Console UI— and then restart the notebook instance when the job is finished to complete the example. If you do this, you'll need to rerun the import and config cells at the start of the notebook before proceeding.
</div>

To monitor the logs for a training job while it is running, click on the links output to the notebook when you start the training job.  You can also visit the [Vertex AI tab in the Cloud Console](https://console.cloud.google.com/vertex-ai/training/custom-jobs) for your 'native' GCP project, and click on 'Training', then 'CUSTOM JOBS'.  From that list of jobs, click in to any of them— look for your username— then click on the 'Logs' link in the detailed view.
<img src="https://storage.googleapis.com/amy-jo/images/terra/CleanShot%202022-02-18%20at%2013.53.16%402x.png" width="90%"/>

### About the ML task and dataset

This notebook shows an example of training an _image classification_ [Keras](https://keras.io/) model.

The [PatchCamelyon benchmark](https://www.tensorflow.org/datasets/catalog/patch_camelyon) consists of 327.680 color images (96 x 96px) extracted from histopathologic scans of lymph node sections. Each image is annotated with a
binary label indicating presence of metastatic tissue. 

The model uses one of Keras' prebuilt model architectures, [Xception](https://keras.io/api/applications/xception/). The training does [_transfer learning_](https://en.wikipedia.org/wiki/Transfer_learning) , bootstrapping from model weights trained on the ['imagenet'](https://en.wikipedia.org/wiki/ImageNet) dataset.

<img src="https://storage.googleapis.com/tfds-data/visualization/fig/patch_camelyon-2.0.0.png" width="60%">

## Config and setup

We'll first do some configuration and set some variables.

In [ ]:
import json
import os
import time
from datetime import datetime

import tensorflow as tf
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip

IMAGE_HEIGHT = 96
IMAGE_WIDTH = 96

IMAGE_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)

LABELS = ["non_metastatic", "metastatic"]

BATCH_SIZE = 32
NB_NUM = "02-2"

print(tf.__version__)

We'll set some variables using Workspace Data.  

In [ ]:
OWNER_EMAIL = ""
USER = ""

if (
    "GOOGLE_PROJECT" in os.environ
):  # This env var is set when running in a Terra workspace
    from firecloud import api as fapi

    WORKSPACE_NAME = os.environ["WORKSPACE_NAME"]
    WORKSPACE_NAMESPACE = os.environ["WORKSPACE_NAMESPACE"]
    OWNER_EMAIL = os.environ["OWNER_EMAIL"]
    # WORKSPACE_ATTRIBUTES contains key-value pairs from the "Workspace Data" section of the Workspace "Data" tab.
    WORKSPACE_ATTRIBUTES = (
        fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME)
        .json()
        .get("workspace", {})
        .get("attributes", {})
    )

    # set a variable from the workspace attributes
    PYTHON_PACKAGE_GCS_URI_WS = WORKSPACE_ATTRIBUTES["PYTHON_PACKAGE_GCS_URI_WS"]
    print(f"PYTHON_PACKAGE_GCS_URI_WS: {PYTHON_PACKAGE_GCS_URI_WS}")
else:
    print(
        "Not running on Terra: you will need to set some variables manually. See below."
    )

if OWNER_EMAIL:
    USER = OWNER_EMAIL.split("@")[0]

### Set some variables


**Edit the cell below before running it**.  **Replace the values with the ones for your 'native' GCP project** generated when running the `00_pcam_setup.ipynb` notebook.

In [ ]:
PROJECT_ID = "your-project-id"
# The service account you've set up for these Vertex AI examples
TRAINING_SA = "your-sa-name@your-project-id.iam.gserviceaccount.com"
BUCKET_NAME = (
    "your-bucket-name"  # don't include the 'gs://' prefix; that is added below
)

The `USER` value will be used to create Vertex resource and job names, so that you can locate your info more easily in the GCP Cloud Console.

In [ ]:
if USER == "" or USER is None:
    USER = "your-username"  # <-- CHANGE THIS

Make sure `USER` was set correctly:

In [ ]:
print(f"USER: {USER}")

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Ensure that the PROJECT_ID is set correctly and set your region

Ensure that your project ID has been set correctly. This should be the project ID of the ['native' GCP project](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra).  (This is different from the project for your workspace).

In [ ]:
print(PROJECT_ID)
LOCATION = "us-central1"

### Check the service account used for some of the Vertex AI calls

You'll use the service account that you set up in your native GCP project. Ensure that it's set properly.


In [ ]:
TRAINING_SA

### Set a Cloud Storage bucket to use for this example


In [ ]:
BUCKET = f"gs://{BUCKET_NAME}"
print(BUCKET)

Copy the Python package with the training code to your bucket. This is necessary because the package needs to be in a GCS bucket accessible to Vertex AI in your 'native' GCP project.

In [ ]:
PYTHON_PACKAGE_GCS_URI = BUCKET + "/pcam/dist/trainer-0.7.tar.gz"
print(PYTHON_PACKAGE_GCS_URI)

In [ ]:
!gsutil cp $PYTHON_PACKAGE_GCS_URI_WS $PYTHON_PACKAGE_GCS_URI

In [ ]:
!gsutil ls $PYTHON_PACKAGE_GCS_URI

### Initialize the Vertex AI SDK with your project, location, and bucket settings

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET)

## Optional: Create an Experiment for tracking training related metadata

The Vertex AI Experiments API is useful for tracking information about your training runs.  You can retrieve the logged information via a pandas dataframe for analysis and comparison.

We'll start by creating an `Experiment`.  Then, in following sections, we'll define Experiment `runs` and log information about the training jobs to them.

**Note**: if you want to log to the same `Experiment` as used in notebook `01_1_vertex_ai`, **find the `EXPERIMENT_NAME` generated in that notebook** and set it here before running the next cell. This doesn't impact how the examples run, but lets you see how you can log multiple _runs_ to the same Experiment, and compare and analyze them in aggregate.

In [ ]:
EXPERIMENT_NAME = f"{USER}-pcam-{NB_NUM}-{TIMESTAMP}"
print(f"experiment name: {EXPERIMENT_NAME}")
aiplatform.init(experiment=EXPERIMENT_NAME)

## Run a hyperperameter tuning job using Vertex AI

Next, we'll show how to run a hyperparameter tuning job on Vertex AI, using our training code.  We define the parameters that we want to vary during the HP search— in this example, learning rate and batch size— and how the HP tuning algorithm should vary them during its search.  The training code must accept those parameters as input arguments.

You can find more info on hyperparameter tuning [here](https://cloud.google.com/ai-platform-unified/docs/training/using-hyperparameter-tuning).

<div class="alert alert-block alert-warning">
    <b>Using the default config, the HP tuning job will take about 7 hours to run.</b>
</div>

<img src="https://storage.googleapis.com/amy-jo/images/vertex/hptune.png" width="90%"/>


In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

In [ ]:
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-6:latest"
EPOCHS = 2

In [ ]:
ts = int(time.time())

MODEL_DISPLAY_NAME = f"{USER}-pcam_hptune{NB_NUM}-{ts}"

EPOCHS = 2
GCS_WORKDIR = f"gs://{BUCKET_NAME}/{MODEL_DISPLAY_NAME}"

HPT_GCS_MODEL_SAVEDIR = f"{GCS_WORKDIR}/{ts}"
GCS_METRICS_PATH = f"/gcs/{BUCKET_NAME}/{MODEL_DISPLAY_NAME}/metrics/{ts}"
print(f"model savedir: {HPT_GCS_MODEL_SAVEDIR}, GCS_METRICS_PATH: {GCS_METRICS_PATH}")

CMDARGS = [
    "--epochs",
    str(EPOCHS),
    # "--copy-data",
    "--gcs-workdir",
    GCS_WORKDIR,
    "--gcs-model-savedir",
    HPT_GCS_MODEL_SAVEDIR,
    "--gcs-metrics-path",
    GCS_METRICS_PATH,
    "--image-height",
    str(IMAGE_HEIGHT),
    "--image-width",
    str(IMAGE_WIDTH),
    "--ml-task",
    "patchcamelyon",
    "--fine-tune",
    "false",
    "--batch-size",
    "32",
    "--hptune",
]
print(CMDARGS)
print(PYTHON_PACKAGE_GCS_URI)

Now we'll define the specs for the worker pool of nodes, each of which will run a training job, and the dictionary of hyperparams.

In [ ]:
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-highmem-16",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [PYTHON_PACKAGE_GCS_URI],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

pdict = {
    "batch-size": hpt.DiscreteParameterSpec(
        values=[16, 32, 64, 128, 256], scale="linear"
    ),
    "lr": hpt.DiscreteParameterSpec(values=[1e-4, 1e-3, 1e-2, 1e-1], scale="log"),
}

Then, we'll create and run a job using that config:

In [ ]:
custom_job = aiplatform.CustomJob(
    display_name=MODEL_DISPLAY_NAME,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=BUCKET,
)

In [ ]:
# Create and run a HyperparameterTuningJob

hp_job = aiplatform.HyperparameterTuningJob(
    display_name=MODEL_DISPLAY_NAME,
    custom_job=custom_job,
    metric_spec={"accuracy": "maximize"},
    parameter_spec=pdict,
    max_trial_count=32,
    parallel_trial_count=4,
    search_algorithm=None,
)

In [ ]:
# ensure your service account is set correctly
TRAINING_SA

In [ ]:
hp_job.run(
    sync=False,
    service_account=TRAINING_SA,
)

## Run a multi-node distributed training job using Vertex AI


In notebook `02_1_vertex_ai`, we ran a single-node, multi-GPU, distributed training job.
Here, we'll show how to run a multi-node distributed training job, on a cluster that Vertex AI sets up for you.  

For this example, this training job will run more slowly than the single-node distributed example above, due to greater network latency, especially since we're setting up each node to use just one GPU. However, for larger jobs it can often make sense to distribute the training across multiple nodes.

The training run for this section will take ~1.5 hours using the default config.

Log a new 'run' in the 'Experiment' that we set up earlier.

In [ ]:
aiplatform.start_run("run-multinode-distrib")

Define some variables that will help us define the training job.

Set the path to the training input data, which is in a GCS bucket, using [GCSFuse](https://cloud.google.com/blog/products/ai-machine-learning/cloud-storage-file-system-ai-training) syntax. We'll pass this path as an input to the training code. This will allow the training job to treat the input data directories as if they are on a local file system.

In [ ]:
ts = int(time.time())

MODEL_DISPLAY_NAME = f"{USER}-pcam_distrib_mw{NB_NUM}-{ts}"

EPOCHS = 2
BATCH_SIZE = 32
GCS_WORKDIR = f"gs://{BUCKET_NAME}/{MODEL_DISPLAY_NAME}"

DISTRIB_GCS_MODEL_SAVEDIR = f"{GCS_WORKDIR}/{ts}"
# DISTRIB_GCS_MODEL_SAVEDIR = 'AIP_MODEL_DIR' # indicate to use Vertex AI-generated dir

DISTRIB_GCS_METRICS_PATH = f"/gcs/{BUCKET_NAME}/{MODEL_DISPLAY_NAME}/metrics/{ts}"
print(
    f"model savedir: {DISTRIB_GCS_MODEL_SAVEDIR}, DISTRIB_GCS_METRICS_PATH: {DISTRIB_GCS_METRICS_PATH}"
)

CMDARGS = [
    "--epochs",
    str(EPOCHS),
    "--batch-size",
    str(BATCH_SIZE),
    # "--copy-data",
    "--multi-node",
    "--gcs-workdir",
    GCS_WORKDIR,
    "--gcs-model-savedir",
    DISTRIB_GCS_MODEL_SAVEDIR,
    "--gcs-metrics-path",
    DISTRIB_GCS_METRICS_PATH,
    "--image-height",
    str(IMAGE_HEIGHT),
    "--image-width",
    str(IMAGE_WIDTH),
    "--ml-task",
    "patchcamelyon",
]
print(CMDARGS)

In [ ]:
argslist = CMDARGS.copy()
argslist.insert(argslist.index("--multi-node") + 1, "True")
args_dict = {argslist[i]: argslist[i + 1] for i in range(0, len(argslist), 2)}
print(args_dict)

In [ ]:
PARAMS = {"model_display_name": MODEL_DISPLAY_NAME}
PARAMS = {**PARAMS, **args_dict}
print(PARAMS)

In [ ]:
aiplatform.log_params(PARAMS)

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_T4, 2)
TRAIN_COMPUTE = "n1-highmem-16"
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-6:latest"

This job uses a 'package', not a script.

In [ ]:
PYTHON_PACKAGE_GCS_URI

In [ ]:
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=MODEL_DISPLAY_NAME,
    python_package_gcs_uri=PYTHON_PACKAGE_GCS_URI,
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
)

In [ ]:
model2 = job.run(
    args=CMDARGS,
    replica_count=3,
    machine_type=TRAIN_COMPUTE,
    accelerator_type=TRAIN_GPU.name,
    accelerator_count=TRAIN_NGPU,
    sync=False,
)

For this job, the model is just saved to GCS (and not automatically uploaded to Vertex AI).  You can find it here: `DISTRIB_GCS_MODEL_SAVEDIR`.

#### Retrieve and save the training metrics to the Experiments `run` info

After the training job finishes, you can download and log the metrics information to the Experiment `run`.

**Wait until training has completed** to run this section.

In [ ]:
DISTRIB_GCS_METRICS_PATH

In [ ]:
metrics_file = f"{DISTRIB_GCS_METRICS_PATH}/metrics.json".replace("/gcs/", "gs://")
metrics_file

In [ ]:
!gsutil cat $metrics_file > temp_metrics.json
fp = open("temp_metrics.json")
metrics = json.load(fp)
_ = metrics.pop("all_labels")
_ = metrics.pop("all_preds")

In [ ]:
metrics = {k: float(v) for k, v in metrics.items()}
metrics

Ensure we're using the correct 'run' context within the Experiment:

In [ ]:
aiplatform.start_run("run-multinode-distrib")

Log the metrics info to the run:

In [ ]:
aiplatform.log_metrics(metrics)

In [ ]:
dataframe = aiplatform.get_experiment_df(experiment=EXPERIMENT_NAME)
dataframe

## Cleanup

For the examples in this notebook, the model was not automatically uploaded to Vertex AI, so you don't need to delete it.

The training instances are automatically torn down after the job completes. 

If the GCS bucket that you used is not set to automatically delete old files, then you can clean up your GCS bucket as well.  An easy way to do this is via the [Cloud Console UI](https://pantheon.corp.google.com/storage/browser).


In [ ]:
# Delete the Experiment
# This code requires google-cloud-aiplatform >=1.8
c = aiplatform.metadata._Context(EXPERIMENT_NAME)
c.delete()

## Provenance

In [ ]:
import datetime

print(datetime.datetime.now())

In [ ]:
!pip3 freeze

--------------------------------
Copyright 2021 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style  
license that can be found in the LICENSE file or at  
https://developers.google.com/open-source/licenses/bsd